In [14]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import SABRModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [98]:
# -------
# INPUTS
# -------

# Pricing parameters
F     = 5000
alpha = 0.25
beta  = 1.0
rho   = -0.
nu    = 1.5
r     = 0
atm_bd = 20
tail_bd = 20

# Scenario parameters
scenario_min = -0.3
scenario_max = 0.3
scenario_step = 0.05

option_strikes = [-0.25, -0.2 , -0.15, -0.1 , -0.05, 0.05,  0.1 ,  0.15, 0.2 ,  0.25]

In [99]:
# Scenario parameters
num = int((scenario_max-scenario_min)/0.01 + 1)
slide_to_compute = list(np.round(np.linspace(start=scenario_min, stop=scenario_max, num=num), 2))

# Option portfolio
market_data_list = list()

# Option ATM
K = F
T = atm_bd / 250
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{atm_bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': atm_bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

for strike in option_strikes:
    K = F * (1+strike)
    T = tail_bd / 250
    option_type = 'call' if K >= F else 'put'
    market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
    market_data_list.append({
        'symbol': f"{tail_bd}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': tail_bd, 
        'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [100]:
# ATM Scenarios
atm_market_data = market_data_df.loc[market_data_df['F']==market_data_df['K']]
atm_qty = -50000 / atm_market_data['theta']
atm_scenario = atm_market_data.loc[:, ['theta'] + slide_to_compute]
atm_scenario = atm_scenario.drop(0, axis=1)
atm_scenario.rename(columns={'theta': 0}, inplace=True)
atm_scenario = atm_scenario.sort_index(axis=1)
atm_scenario = atm_scenario.multiply(atm_qty, axis=0)
atm_scenario = atm_scenario.iloc[0].transpose()
bins = np.arange(scenario_min, scenario_max+scenario_step, scenario_step)  # from -0.30 to +0.30, stepping by 0.05
labels = ["{:.0f}% to {:.0f}%".format(bins[i]*100, bins[i+1]*100) for i in range(len(bins)-1)]
atm_scenario_bucket = atm_scenario.groupby(pd.cut(atm_scenario.index, bins=bins, labels=labels), observed=False).sum().reset_index()
atm_scenario_bucket.columns = ["bucket", "sum"]
atm_scenario_bucket = atm_scenario_bucket.set_index('bucket')

In [101]:
target_hedge_portfolio_scenario_bucket = atm_scenario_bucket*-1

In [102]:
# OTM Scenarios
otm_market_data = market_data_df.loc[market_data_df['F']!=market_data_df['K']]
otm_scenario = otm_market_data.loc[:, ['theta'] + slide_to_compute]
otm_scenario = otm_scenario.drop(0, axis=1)
otm_scenario.rename(columns={'theta': 0}, inplace=True)
otm_scenario = otm_scenario.sort_index(axis=1)
otm_scenario = otm_scenario.transpose()
bins = np.arange(scenario_min, scenario_max+scenario_step, scenario_step)  # from -0.30 to +0.30, stepping by 0.05
labels = ["{:.0f}% to {:.0f}%".format(bins[i]*100, bins[i+1]*100) for i in range(len(bins)-1)]

otm_scenario = otm_scenario.copy()  # if you don’t want to modify the original
otm_scenario["bucket"] = pd.cut(
    otm_scenario.index,
    bins=bins,
    labels=labels,
    include_lowest=True
)

# 2) group by that column
otm_scenario_bucket = (
    otm_scenario
    .groupby("bucket", as_index=False, observed=False)   # now "bucket" really is a column
    .sum()
)
otm_scenario_bucket = otm_scenario_bucket.set_index('bucket')

In [103]:
atm_scenario_bucket

,sum
bucket,
-30% to -25%,"38,530,930.50"
-25% to -20%,"29,565,083.04"
-20% to -15%,"17,245,401.21"
-15% to -10%,"13,796,442.65"
-10% to -5%,"6,132,364.81"
-5% to -0%,"1,114,089.69"
-0% to 5%,"544,054.41"
5% to 10%,"4,567,799.01"
10% to 15%,"11,281,389.12"


In [104]:
otm_scenario_bucket

symbol,20T_3750.0K_put,20T_4000.0K_put,20T_4250.0K_put,20T_4500.0K_put,20T_4750.0K_put,20T_5250.0K_call,20T_5500.0K_call,20T_5750.0K_call,20T_6000.0K_call,20T_6250.0K_call
bucket,,,,,,,,,,
-30% to -25%,"1,103.57","2,298.67","3,583.90","4,534.04","4,555.07","1,834.24",790.73,305.11,114.84,44.19
-25% to -20%,265.70,840.96,"1,725.14","2,531.51","2,742.12","1,167.36",507.94,196.97,74.35,28.66
-20% to -15%,57.25,248.64,696.96,"1,256.04","1,516.94",698.52,307.85,120.20,45.55,17.60
-15% to -10%,17.53,90.57,333.07,776.26,"1,109.59",584.15,263.27,104.05,39.72,15.42
-10% to -5%,2.64,15.49,71.73,224.84,416.69,285.05,135.10,54.95,21.35,8.38
-5% to -0%,0.04,0.68,4.86,21.99,57.48,62.34,33.17,14.52,5.92,2.41
-0% to 5%,0.23,1.05,4.19,13.52,30.04,27.48,11.50,3.23,0.57,-0.04
5% to 10%,1.20,5.53,23.11,80.51,204.34,309.06,184.96,80.90,29.87,10.33
10% to 15%,2.32,10.87,46.52,168.15,454.15,896.94,656.49,349.41,150.66,58.14


In [105]:
otm_scenario_bucket.drop(['-5% to -0%', '-0% to 5%'])

symbol,20T_3750.0K_put,20T_4000.0K_put,20T_4250.0K_put,20T_4500.0K_put,20T_4750.0K_put,20T_5250.0K_call,20T_5500.0K_call,20T_5750.0K_call,20T_6000.0K_call,20T_6250.0K_call
bucket,,,,,,,,,,
-30% to -25%,"1,103.57","2,298.67","3,583.90","4,534.04","4,555.07","1,834.24",790.73,305.11,114.84,44.19
-25% to -20%,265.70,840.96,"1,725.14","2,531.51","2,742.12","1,167.36",507.94,196.97,74.35,28.66
-20% to -15%,57.25,248.64,696.96,"1,256.04","1,516.94",698.52,307.85,120.20,45.55,17.60
-15% to -10%,17.53,90.57,333.07,776.26,"1,109.59",584.15,263.27,104.05,39.72,15.42
-10% to -5%,2.64,15.49,71.73,224.84,416.69,285.05,135.10,54.95,21.35,8.38
5% to 10%,1.20,5.53,23.11,80.51,204.34,309.06,184.96,80.90,29.87,10.33
10% to 15%,2.32,10.87,46.52,168.15,454.15,896.94,656.49,349.41,150.66,58.14
15% to 20%,3.51,16.51,71.34,262.13,729.50,"1,680.04","1,425.89",907.66,462.76,202.07
20% to 25%,4.71,22.24,96.58,357.99,"1,012.71","2,551.07","2,389.27","1,753.13","1,055.03",536.43


In [107]:
X = otm_scenario_bucket.drop(['-5% to -0%', '-0% to 5%']).values
b = target_hedge_portfolio_scenario_bucket.drop(['-5% to -0%', '-0% to 5%']).values

w = np.linalg.solve(X, b)



In [108]:
w

array([[ 42906.51950287],
       [-30544.63868064],
       [ 11752.93932749],
       [   261.59635231],
       [ -8000.48422917],
       [-19582.33524585],
       [ 24547.02645314],
       [-26532.77443382],
       [ 21829.21821981],
       [ -9887.21647222]])

In [ ]:

weights = pd.Series(w, index=otm_market_data.index)

print(weights)

In [114]:
otm_scenario = otm_market_data.loc[:, ['theta'] + slide_to_compute]
otm_scenario = otm_scenario.drop(0, axis=1)
otm_scenario.rename(columns={'theta': 0}, inplace=True)
otm_scenario = otm_scenario.sort_index(axis=1)
otm_scenario = otm_scenario.multiply(weights, axis=0)

In [115]:
total_scenario = atm_scenario.sum() + otm_scenario.sum()

In [116]:
px.line(total_scenario)